In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from auctioneer import *
from train_crypto_bot import *

2022-12-07 08:44:49.854265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
s3 = create_s3()

data = load_data(s3, START_DATE, END_DATE)

08:45:13: Reading data
08:45:21: Original shape: (826561, 3)
08:45:21: After clipping start date: (462210, 3)
08:45:21: Original shape: (462210, 3)
08:45:21: After clipping end date: (4643, 3)


In [3]:
X_train, Y_train, X_test, Y_test, scalers = window_data(data, HISTORY_STEPS, TARGET_STEPS)

/Users/landon/Projects/auctioneer/src/auctioneer.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = norm
/Users/landon/Projects/auctioneer/src/auctioneer.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col] = norm
/Users/landon/Projects/auctioneer/src/auctioneer.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [4]:
BATCH_SIZE = 8
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
val_dataset = val_dataset.batch(BATCH_SIZE)

2022-12-07 08:45:22.415226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = encoder_model(HISTORY_STEPS, TARGET_STEPS, 3)

In [6]:
# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.Huber()

# Prepare the metrics.

train_acc_metric = tf.keras.metrics.MeanAbsoluteError()
val_acc_metric = tf.keras.metrics.MeanAbsoluteError()

In [8]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [10]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * BATCH_SIZE))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training loss (for one batch) at step 0: 2.3239
Seen so far: 8 samples
Training loss (for one batch) at step 200: 1.1245
Seen so far: 1608 samples
Training loss (for one batch) at step 400: 0.9420
Seen so far: 3208 samples
Training acc over epoch: 2.6134
Validation acc: 1.5381
Time taken: 87.86s

Start of epoch 1
Training loss (for one batch) at step 0: 2.9981
Seen so far: 8 samples
Training loss (for one batch) at step 200: 1.0929
Seen so far: 1608 samples
Training loss (for one batch) at step 400: 0.9904
Seen so far: 3208 samples
Training acc over epoch: 1.9987
Validation acc: 1.6411
Time taken: 79.08s
